Code for ingestion of the dataset

Code for exploration of the dataset

The code should be organized logically with appropriate comments

Best practice: someone who is completely unfamiliar with your project and code can easily understand what you have done and run the code error-free. 

The code must run error-free.

Clearly mark where the week starts and ends.

In [1]:
# Code for ingestion of the dataset

import time
import requests
import pandas as pd
import yfinance as yf
from pathlib import Path


# Locate repo root and Data folder

def find_repo_root(start_path=None):
    """Walk up directories until .git is found"""
    p = Path(start_path or Path.cwd()).resolve()
    for parent in [p] + list(p.parents):
        if (parent / ".git").exists():
            return parent
    # fallback to current working directory
    return p

REPO_ROOT = find_repo_root()
DATA_DIR = REPO_ROOT / "Data"
DATA_DIR.mkdir(parents=True, exist_ok=True)

# Output file (csv inside a zip)
output_file = "stock_symbols_tongyu_testing.csv.zip"
output_path = DATA_DIR / output_file


# SEC Parameters

SEC_URL = "https://www.sec.gov/files/company_tickers_exchange.json"
SEC_HEADERS = {"User-Agent": "YourAppName your_email@example.com"}  # required by SEC

def get_universe_from_sec(limit):
    r = requests.get(SEC_URL, headers=SEC_HEADERS, timeout=30)
    r.raise_for_status()
    j = r.json()
    df = pd.DataFrame(j["data"], columns=j["fields"])
    df = df.rename(columns={"ticker": "symbol", "name": "company_name"})
    df["symbol"] = df["symbol"].str.upper()
    return df[["symbol", "company_name"]].drop_duplicates()

def yf_fetch_info(symbol: str) -> dict:
    candidates = [symbol, symbol.replace("-", ".")]
    for sym in candidates:
        try:
            t = yf.Ticker(sym)
            info = t.get_info()
            if info and isinstance(info, dict) and info.get("quoteType") in ("EQUITY", "ETF"):
                return info
        except Exception:
            pass
    return {}

def build_base_table(limit, sleep_s=0.35):
    universe = get_universe_from_sec(limit=limit)
    print("Symbols pulled:", len(universe.index))
    rows = []
    count = 0
    for sym in universe["symbol"].tolist():
        rows.append(yf_fetch_info(sym))
        count += 1
        time.sleep(sleep_s)
        if count % 200 == 0:
            print(f"{count} rows completed")

    facts = pd.DataFrame(rows)

    df_cols = list(facts.columns)
    added = ["symbol", "company_name"]
    cols = df_cols + added

    base = (
        universe
        .merge(facts, on="symbol", how="left")
        [cols]
        .drop_duplicates(subset=["symbol"])
    )
    return base

# -------------------------------
# Run + Save
# -------------------------------
df_base = build_base_table(limit=1000, sleep_s=0.35)

print(df_base.head(10))
print(df_base.shape)
print(df_base.columns)

# Save as CSV inside a zip
df_base.to_csv(output_path, index=False, compression="zip")
print(f"Saved: {output_path}")

# Load later (same folder)
df = pd.read_csv(output_path, compression="zip")
grouped = df.groupby("sector")["sector"].count()
print(grouped)

Symbols pulled: 10317
200 rows completed
400 rows completed
600 rows completed
800 rows completed
1000 rows completed
1200 rows completed
1400 rows completed
1600 rows completed


HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: WNS"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: WNS"}}}


1800 rows completed
2000 rows completed
2200 rows completed
2400 rows completed
2600 rows completed


HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: TIXT"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: TIXT"}}}


2800 rows completed
3000 rows completed
3200 rows completed
3400 rows completed
3600 rows completed
3800 rows completed
4000 rows completed
4200 rows completed
4400 rows completed
4600 rows completed


...

HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: KPHMW"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: KPHMW"}}}


9000 rows completed
9200 rows completed


HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: SKYH-WT"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: SKYH.WT"}}}


9400 rows completed
9600 rows completed
9800 rows completed
10000 rows completed
10200 rows completed
                    address1           city state         zip        country  \
0  2788 San Tomas Expressway    Santa Clara    CA       95051  United States   
1  1600 Amphitheatre Parkway  Mountain View    CA       94043  United States   
2         One Apple Park Way      Cupertino    CA       95014  United States   
3          One Microsoft Way        Redmond    WA  98052-6399  United States   
4     410 Terry Avenue North        Seattle    WA  98109-5210  United States   
5                 1 Meta Way     Menlo Park    CA       94025  United States   
6          3421 Hillview Ave      Palo Alto    CA       94304  United States   
7               1 Tesla Road         Austin    TX       78725  United States   
8         3555 Farnam Street          Omaha    NE       68131  United States   
9     Lilly Corporate Center   Indianapolis    IN       46285  United States   

            phone

/tmp/ipykernel_1281938/9621364.py:96: DtypeWarning: Columns (202,209,211,212,213,214) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(output_path, compression="zip")
